## Heat equation and the Poisson problem

In [1]:
import mfem.ser as mfem
import numpy as np
import time
from glvis import glvis

In [2]:
order = 2
mesh = mfem.Mesh(30, 30, "TRIANGLE")
dim = mesh.Dimension()

fec = mfem.H1_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)

print("Number of unknowns: " + str(fes.GetVSize()))

def s1d(k, x):
    return np.sin(2*k*np.pi*x)
def u1d(k, x):
    skx = s1d(k, x)
    if skx == 0.0:
        return 0.0
    else:
        return np.exp(-1.0 / (skx**2)) * np.sign(skx)
def w1d(n, x):
    val = 0.0
    for j in range(0, n):
        val += u1d(3**j, x)
    return val

class InitialCondition(mfem.PyCoefficient):
    def EvalValue(self, xvec):
        val = 1.0
        for x in xvec:
            val *= w1d(4, x)
        return val

ic_coeff = InitialCondition()

u = mfem.GridFunction(fes)
u.ProjectCoefficient(ic_coeff)

Number of unknowns: 3721


In [3]:
glvis((mesh, u))

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [4]:
# 6. Set up and assemble the bilinear and linear forms corresponding to 
#    the DG discretization. The DGTraceIntegrator involves integrals over
#    mesh interior faces.

one = mfem.ConstantCoefficient(1.0)

m = mfem.BilinearForm(fes)
m.AddDomainIntegrator(mfem.MassIntegrator())
k = mfem.BilinearForm(fes)
k.AddDomainIntegrator(mfem.DiffusionIntegrator())
b = mfem.LinearForm(fes)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(one))

m.Assemble()
m.Finalize()
k.Assemble()
k.Finalize()
b.Assemble()

ess_dofs = mfem.intArray()
ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
fes.GetEssentialTrueDofs(ess_bdr, ess_dofs)

M = mfem.OperatorPtr()
K = mfem.OperatorPtr()
m.FormSystemMatrix(ess_dofs, M)
k.FormSystemMatrix(ess_dofs, K)


### Solve and Plot the Solution with GLVis

In [5]:
u.ProjectCoefficient(ic_coeff)

g = glvis((mesh, u))
g.render()

class FE_Evolution(mfem.PyTimeDependentOperator):
    def __init__(self, M, K, b):
        mfem.PyTimeDependentOperator.__init__(self, M.Size())
        self.K = K        
        self.M = M
        self.b = b
        self.z = mfem.Vector(M.Size())
        self.zp = np.zeros(M.Size())
        self.M_prec = mfem.DSmoother()        
        self.M_solver = mfem.CGSolver()
        self.M_solver.SetPreconditioner(self.M_prec)        
        self.M_solver.SetOperator(M)
        self.M_solver.iterative_mode = False
        self.M_solver.SetRelTol(1e-9)
        self.M_solver.SetAbsTol(0.0)
        self.M_solver.SetMaxIter(1000)
        self.M_solver.SetPrintLevel(0)
        
    def Mult(self, x, y):
        self.K.Mult(x, self.z)
        self.z *= -1.0
        self.z += b
        self.M_solver.Mult(self.z, y)

heat_eq = FE_Evolution(mfem.OperatorHandle2SparseMatrix(M), mfem.OperatorHandle2SparseMatrix(K), b)

ode_solver = mfem.RK4Solver()
ode_solver.Init(heat_eq)

t = 0.0
ti = 0
dt = 2e-5
t_final = 1e-1
vis_steps = 5

time.sleep(1)
while True:
    if t > t_final - dt/2: break
    t, dt = ode_solver.Step(u, t, dt);
    ti = ti + 1
    if ti % vis_steps == 0:
        g.update((mesh, u))
        time.sleep(0.05)
        print(f"time step: {ti}, time: {t:.2e}", end="\r")

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…

In [6]:
u.Assign(0.0)
U = mfem.Vector()
B = mfem.Vector()
k.FormLinearSystem(ess_dofs, u, b, K, U, B)

# solve it using PCG solver and store the solution to x
GS = mfem.GSSmoother(mfem.OperatorHandle2SparseMatrix(K))
mfem.PCG(K, GS, B, U, 1, 200, 1e-12, 0.0)
k.RecoverFEMSolution(U, b, u)

glvis((mesh, u))

   Iteration :   0  (B r, r) = 0.000156583
   Iteration :   1  (B r, r) = 0.000983425
   Iteration :   2  (B r, r) = 0.000986053
   Iteration :   3  (B r, r) = 0.00122757
   Iteration :   4  (B r, r) = 0.000895496
   Iteration :   5  (B r, r) = 0.000667813
   Iteration :   6  (B r, r) = 0.000700479
   Iteration :   7  (B r, r) = 0.000521907
   Iteration :   8  (B r, r) = 0.000401969
   Iteration :   9  (B r, r) = 0.000401193
   Iteration :  10  (B r, r) = 0.00027078
   Iteration :  11  (B r, r) = 0.000194821
   Iteration :  12  (B r, r) = 0.000184164
   Iteration :  13  (B r, r) = 0.000143731
   Iteration :  14  (B r, r) = 9.08703e-05
   Iteration :  15  (B r, r) = 6.59717e-05
   Iteration :  16  (B r, r) = 4.44696e-05
   Iteration :  17  (B r, r) = 2.69184e-05
   Iteration :  18  (B r, r) = 1.38891e-05
   Iteration :  19  (B r, r) = 6.23162e-06
   Iteration :  20  (B r, r) = 1.89955e-06
   Iteration :  21  (B r, r) = 5.54674e-07
   Iteration :  22  (B r, r) = 5.77362e-07
   Iteration 

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…